<a href="https://colab.research.google.com/github/jparrett/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Tools_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [6]:
#Create dataframe of items with total votes over 20
over20_df = df.filter("total_votes>=20")
over20_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

In [17]:
# Create a helpful votes dataframe
helpful_df = over20_df.filter(over20_df['helpful_votes']/over20_df['total_votes']>=.5)
helpful_df.show(5)
helpful_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

31830

In [23]:
# Create a dataframe of paid reviews
paid_df = helpful_df.filter(helpful_df["vine"] == "Y")
paid_df.show(5)
paid_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21084288|R3QIQRG107AP19|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|           38|         41|   Y|                N|Essential home ow...|We are not pros. ...| 2015-08-27|
|         US|   52512530|R1ZNBHGCYE8VN1|B00OJ72LHK|     369815253|BLACK+DECKER BDEJ...|           Tools|          5|    

285

In [24]:
unpaid_df = helpful_df.filter(helpful_df["vine"] == "N")
unpaid_df.show(5)
unpaid_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   47749608| RXAHWIC1584UQ|B00NKSMPZW|     824618679|ClearArmor 141001...|           Tools|          5|           20|         23|   N|                Y|          Five Stars|geart product :) ...| 2015-08-31|
|         US|   53082915| RSBELAIC899DO|B00LPFIPPQ|     848632692|WEN 6510 Oscillating|           Tools|          5|    

31545

In [28]:
#total number of reviews
print("Paid Reviews: %s" % paid_df.count())

#number of 5-star reviews
print("5-star Paid Reviews:  %s" % paid_df.filter("star_rating==5").count())

#% of 5 star reviews
print("Percent of 5-star Paid Reviews:  %s" % "{:.2%}".format(paid_df.filter("star_rating==5").count()/paid_df.count()))



Paid Reviews: 285
5-star Paid Reviews:  163
Percent of 5-star Paid Reviews:  57.19%


In [29]:
#total number of reviews
print("Unpaid Reviews: %s" % unpaid_df.count())

#number of 5-star reviews
print("5-star Unpaid Reviews:  %s" % unpaid_df.filter("star_rating==5").count())

#% of 5 star reviews
print("Percent of 5-star Unpaid Reviews:  %s" % "{:.2%}".format(unpaid_df.filter("star_rating==5").count()/unpaid_df.count()))

Unpaid Reviews: 31545
5-star Unpaid Reviews:  14614
Percent of 5-star Unpaid Reviews:  46.33%
